<a href="https://colab.research.google.com/github/andre-arantes/ia/blob/master/tp1/etapa5/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
with open('/content/sample_data/PreProcessamento.pkl', 'rb') as f:
  X_trainscaled, X_testscaled, y_treino, y_teste = pickle.load(f)

In [ ]:
# Importando bibliotecas
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np

In [ ]:
modelo = RandomForestClassifier(random_state = 0)
from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(modelo.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}


#Random Grid Search

Para montagem do RandomGrid, foi utilizado os seguintes parametros: quantidade de árvores (40 a 400), cálculo para quantidade de atributos (log2 ou sqrt), profundidade máxima da árvore (1 a 20 ou sem limite), mínimo de samples para dividir um nó (2 3 ou 5), numero mínimo de samples em cada folha da árvore (1, 2 ou 4) e uso de bootstrap (verdadeiro ou falso).

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 40, stop = 400, num = 10)]
# Number of features to consider at every split
max_features = ['log2', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 20, num = 3)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [1, 10, 20, None],
 'max_features': ['log2', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [40, 80, 120, 160, 200, 240, 280, 320, 360, 400]}


**RandomSerach com cross validation**

RandomizedSearchCV com o RandomGrid como um de seus parâmetros, além do kfold (5) percorrendo 100 combinações e com o scoring buscando o melhor recall da classe 1.

In [ ]:

from sklearn.metrics import make_scorer, recall_score


# Create a custom scoring function for recall of class 1
scoring = {'Recall_Class1': make_scorer(recall_score, pos_label=1)}

# Random search of parameters, using 5 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=modelo, param_distributions=random_grid,
                              n_iter = 100, scoring=scoring,refit='Recall_Class1',
                              cv = 5, verbose=2, random_state=0, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(X_trainscaled, y_treino)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [1, 10, 20, None],
                                        'max_features': ['log2', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [40, 80, 120, 160, 200,
                                                         240, 280, 320, 360,
                                                         400]},
                   random_state=0, refit='Recall_Class1',
                   return_train_score=True,
                   scoring={'Recall_Class1': make_scorer(recall_score, pos_label=1)},
                   verbose=2)

**Avaliação do modelo base**

A avaliação do modelo é feita a partir do recall da classe verdadeira (tem diabetes)

In [ ]:
def evaluate(model, test_features, test_labels):
    previsoes = model.predict(test_features)
    cr = classification_report(test_labels, previsoes, output_dict=True)['1']['recall']
    return cr

In [ ]:
base_model = RandomForestClassifier(random_state = 0)
base_model.fit(X_trainscaled, y_treino)
base_recall = evaluate(base_model, X_testscaled, y_teste)
base_recall

0.7857935627081021

**avaliação do melhor modelo**

In [ ]:
best_random = rf_random.best_estimator_
random_recall = evaluate(best_random, X_testscaled, y_teste)
random_recall

0.7846836847946725

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_recall - base_recall) / base_recall))

Improvement of -0.14%.


In [ ]:
rf_random.best_params_

{'n_estimators': 240,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'log2',
 'max_depth': None,
 'bootstrap': True}

In [ ]:
rf_random.cv_results_

{'mean_fit_time': array([0.87572865, 0.83504472, 4.49728818, 1.40925651, 0.35229082,
        3.07021966, 5.00599918, 3.31931334, 4.20134392, 5.19112744,
        5.20230899, 1.77885861, 0.84815784, 2.94438782, 0.63657556,
        3.979459  , 3.68627996, 2.03170066, 2.83805676, 4.82466769,
        1.31759853, 1.78099346, 1.24242225, 1.4812963 , 6.01319942,
        0.59267621, 0.52776117, 2.20697293, 2.32871723, 1.40229516,
        0.34118443, 1.67106695, 2.98849511, 4.24720254, 4.44259672,
        1.63033252, 4.50130267, 1.2989089 , 2.15348535, 3.61597567,
        1.29673395, 0.55874739, 3.95454893, 1.22098789, 4.48726101,
        3.60056448, 2.13573847, 3.0966032 , 0.49378195, 1.49122858,
        1.18996611, 4.83609338, 0.28599367, 1.75645924, 0.34411273,
        3.39963107, 4.84682264, 1.99736443, 6.36231976, 2.50006409,
        0.53987026, 1.70137239, 2.39057922, 0.16103525, 1.04819822,
        4.37375278, 0.71312814, 4.45629911, 4.48808541, 1.71309686,
        1.06097035, 2.28488116,

#Resultados

In [ ]:
previsoes = best_random.predict(X_testscaled)
print(classification_report(y_teste, previsoes))
report_random_forest = classification_report(y_teste, previsoes, output_dict=True)
relatorio_random_forest = report_random_forest['1']['recall']

              precision    recall  f1-score   support

           0       0.99      0.93      0.96     17177
           1       0.38      0.78      0.51       901

    accuracy                           0.93     18078
   macro avg       0.68      0.86      0.74     18078
weighted avg       0.96      0.93      0.94     18078



**Serializar Classification Report**

In [ ]:
import pickle
with open('/content/sample_data/relatorioRandomForest.pkl', mode = 'wb') as f:
  pickle.dump([relatorio_random_forest], f)